In [23]:
from dotenv import dotenv_values
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import TextLoader
from langchain.embeddings import CacheBackedEmbeddings
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from pathlib import Path


config = dotenv_values(".env")

chat = ChatOpenAI(
    openai_api_key=config["OPENAI_API_KEY"],
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# https://python.langchain.com/docs/versions/migrating_memory/
memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=20,
    return_messages=True,
    memory_key="history",
)

cache_path = "./.cache"
embedding_path = f"{cache_path}/challenge-04"
Path(embedding_path).mkdir(parents=True, exist_ok=True)
embedding_dir = LocalFileStore(embedding_path)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./challenge-04.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(
    openai_api_key=config["OPENAI_API_KEY"],
)
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    embedding_dir,
)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context.\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | chat
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    # print(result.content)

In [19]:
invoke_chain("Aaronson 은 유죄인가요? (Is Aaronson guilty?)")

Aaronson은 유죄가 아닙니다. 그는 죄를 지은 적이 없으며, 그에 대한 유죄 판결은 잘못된 것입니다.The human asks if Aaronson is guilty. The AI responds that Aaronson is not guilty, stating that he has never committed a crime and that the guilty verdict against him is incorrect.

In [20]:
invoke_chain(
    "그가 테이블에 어떤 메시지를 썼나요? (What message did he write in the table?)"
)

그는 테이블에 다음과 같은 메시지를 썼습니다: "FREEDOM IS SLAVERY" (자유는 노예다) 그리고 그 아래에 "TWO AND TWO MAKE FIVE" (둘과 둘은 다섯이다)라고 썼습니다.The human asks if Aaronson is guilty. The AI responds that Aaronson is not guilty, stating that he has never committed a crime and that the guilty verdict against him is incorrect. The human then inquires about a message Aaronson wrote on the table, to which the AI reveals that he wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE."

In [21]:
invoke_chain("Julia 는 누구인가요? (Who is Julia?)")

Julia는 Winston의 사랑하는 사람으로, 그와 함께 자유를 추구하고 반당파적인 관계를 맺었던 인물입니다. 그녀는 Winston과 함께 당의 억압에 맞서 싸우려 했지만, 결국 그들의 관계는 당의 감시와 압박에 의해 파괴됩니다.The human asks if Aaronson is guilty. The AI responds that Aaronson is not guilty, stating that he has never committed a crime and that the guilty verdict against him is incorrect. The human then inquires about a message Aaronson wrote on the table, to which the AI reveals that he wrote "FREEDOM IS SLAVERY" and "TWO AND TWO MAKE FIVE." The human then asks about Julia, and the AI explains that Julia is Winston's beloved, who sought freedom alongside him and had a rebellious relationship, but their bond was ultimately destroyed by the Party's surveillance and pressure.